In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy import linalg
from scipy.optimize import LinearConstraint
from scipy.optimize import NonlinearConstraint
from math import comb
import random
import time
from sklearn.metrics import mean_squared_error
import copy
import scipy.linalg as lin

import torch
import torch.nn as nn
from scipy.optimize import brute

import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

import time as timer

# Functions

In [ ]:
#Legendre domain
class legendre:
    def __init__(self,theta, dim, dt): 
        self.theta = theta # Representation window
        self.dim = dim # Order of representation
        self.dt = dt # Sampling time
        
        self.P = self.init_P()
        
    def init_P(self):
        window = int(self.theta/self.dt)
        P = np.zeros((window+1,self.dim))
        for t in range(window+1):
            r = t / window
            for i in range(self.dim):
                temp_c = pow((-1),i)
                temp_s = 0
                for j in range(i+1):
                    temp_s = temp_s + comb(i,j) * comb((i+j),j) * pow((-r),j)
             
                P[t,i] = temp_s*temp_c
        return P
    
    def decode(self,M): #Returns time domain data given legendre coefficients M
        window = int(self.theta/self.dt)
        u = np.zeros(window)
        for t in range(window):
            u[t] = np.sum(np.multiply(self.P[t,:],M))
        return u
    
    def encode(self, u): #Returns legendre coefficients M given time domain data u
        a = np.zeros(self.dim)
        for n in range(self.dim):
            for k in range(int(self.theta/self.dt)):
                a[n] = a[n] + self.P[k,n]*u[k]*self.dt
            a[n] = ( (1/self.theta)*(2*n+1) ) * a[n]
        return a
        
class cart_pole:
    def __init__(self,g,l,mp,mc,dt):
        self.g = g #Gravity
        self.l = l #Length of pendulum
        self.mp = mp #Mass of pendulum
        self.mc = mc #Mass of cart
        self.dt = dt #Sampling time
        
    def dynamics(self,x,u):
        x_dot = np.zeros(4)
        x_dot[0] = x[0] + x[1]*self.dt 

        x_dot[1] = x[1] + ( (u + self.mp * np.sin(x[2]) * (self.l * x[3]**2 - self.g * np.cos(x[2]))) / (self.mc + self.mp * np.sin(x[2])**2) )*(self.dt)
        
        x_dot[2] = x[2] + x[3]*self.dt
        
        x_dot[3] = x[3] + ( (-u*np.cos(x[2]) - self.mp*self.l*x[3]**2 * np.sin(x[2]) * np.cos(x[2]) + (self.mc + self.mp) * self.g * np.sin(x[2])) / (self.l * (self.mc + self.mp * np.sin(x[2])**2)) )*(self.dt)

        return x_dot

# Collect Data

In [ ]:
#Simulation parameters
num_samples = 100000000
dt = 0.001 #Sample Time
Np = 0.1 #Prediction Window
N = 10 #Order of representation

lower_control_bound = -500
upper_control_bound = 500


#Define system
g = 9.8 #gravity
l = 0.5 #length
dt = 0.001 #sampling time
mp = 0.1 #pole mass
mc = 1 #cart mass
sys = cart_pole(g,l,mp,mc,dt)
leg = legendre(Np,N,dt)


#Create IC
x0 = np.arange(-10, 10.5, 0.50)
x1 = np.arange(-20, 21, 1)
x2 = np.arange(-3.2, 3.20, 0.10)
x3 = np.arange(-10, 10,0.5)

# Create the meshgrid
X0,X1,X2,X3 = np.meshgrid(x0, x1,x2,x3)

# Combine the coordinates into pairs
stacked_array = np.column_stack([X0.ravel(), X1.ravel(), X2.ravel(), X3.ravel()])
IC = stacked_array

num_IC = len(IC)
window = int(Np/dt)
M_u_data = np.zeros((num_samples,N+4))
M_x_data = np.zeros((num_samples,N))
for ds in range(num_samples):
    #Select a initial condition
    IC_index = int(ds*num_IC/num_samples)
    x0 = IC[IC_index]
        
    x_traj = np.zeros((window,4))
    M_u = np.random.uniform(-lower_control_bound,upper_control_bound,N)
    x_traj[0,:] = x0
    u_traj = leg.decode(M_u)

    #Find dynamics from given U trajectory
    for k in range(1,window):
         x_traj[k,:] = sys.dynamics(x_traj[k-1,:],u_traj[k-1])
    M_x = leg.encode(x_traj[:,2])
    
    M_u_data[ds,0:4] = x0
    M_u_data[ds,4:N+4] = M_u
    M_x_data[ds,:] = M_x

    #To display progess
    if (ds+1)%int(num_samples/10) == 0:
        print(str(100*(ds+1)/num_samples)+"%")
#Save data
np.savetxt('M_u_data_CartPole.txt', M_u_data)    
np.savetxt('M_x_data_CartPole.txt', M_x_data)
    
    

# Train Model

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(SimpleNN, self).__init__()
        self.layer1 = nn.Linear(input_size, 500)
        self.layer2 = nn.Linear(500, output_size)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.layer2(x)
        return x
        
M_u_data = np.loadtxt('M_u_data_CartPole.txt')
M_x_data = np.loadtxt('M_x_data_CartPole.txt')

num_samples = M_u_data.shape[0]
dim = M_u_data.shape[1]

#Load in training data
inputs = torch.tensor(M_u_data, dtype=torch.float32)
outputs = torch.tensor(M_x_data, dtype=torch.float32)
dataset = TensorDataset(inputs, outputs)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

#Load model on to device
model = SimpleNN(input_size=dim, output_size=dim-4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
losses = []

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    for batch_inputs, batch_outputs in dataloader:
        batch_inputs, batch_outputs = batch_inputs.to(device), batch_outputs.to(device)
        optimizer.zero_grad()  # Zero the parameter gradients
        predictions = model(batch_inputs)  # Forward pass
        loss = criterion(predictions, batch_outputs)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update parameters
        
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

torch.save(model.state_dict(), f'model_cartPole.pth')